# Core lib

> core classes & helpers

In [ ]:
#| default_exp models.core

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2
from nbdev.showdoc import *

In [ ]:
#| export
import torch.nn as nn
import torch

from abc import ABC, abstractmethod
from nimrod.utils import logger

from torchmetrics import Accuracy


In [ ]:
#| export

class Classifier(ABC):
    def __init__(self, num_classes:int=10, lr:float=1e-3, **kwargs):
        logger.info("Classifier init: num_classes: {}, lr: {}".format(num_classes, lr))
        super().__init__(**kwargs)
        self.save_hyperparameters()
        self.loss = nn.CrossEntropyLoss()
        self.accuracy = Accuracy(task="multiclass", num_classes=num_classes)
        self.lr = lr
    
    @abstractmethod
    def configure_optimizers(self):
        pass
    
    @abstractmethod
    def _step(self, batch, batch_idx):
        pass

    @abstractmethod
    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        pass

    def training_step(self, batch, batch_idx):
        loss, acc = self._step(batch, batch_idx)
        metrics = {"train/loss": loss, "train/acc": acc}
        self.log_dict(metrics, on_epoch=True)
        return loss
    
    def validation_step(self, batch, batch_idx, prog_bar=True, on_step=False, on_epoch=True, sync_dist=True):
        loss, acc = self._step(batch, batch_idx)
        metrics = {"val/loss":loss, "val/acc": acc}
        self.log_dict(metrics, on_step=on_step, on_epoch=on_epoch, sync_dist=sync_dist)
    
    def test_step(self, batch, batch_idx, prog_bar=True, on_step=False, on_epoch=True, sync_dist=True):
        loss, acc = self._step(batch, batch_idx)
        metrics = {"test/loss":loss, "test/acc": acc}
        self.log_dict(metrics, on_step=on_step, on_epoch=on_epoch, sync_dist=sync_dist)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()